<a href="https://colab.research.google.com/github/Dhananjana97/ML-Ride-Fair/blob/master/sec_per_fare_XGBOOST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [460]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# imports

In [0]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

# train set load

In [462]:
train_df = pd.read_csv('/content/drive/My Drive/ML Project/train.csv')
train_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,correct
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,correct
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,correct
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,correct


In [0]:
# def date_time_extractor(d):
#   date_time_columns = ['drop_time', 'pickup_time']
#   for column in date_time_columns:
#     # d.insert(d.columns.get_loc(column), column+'_year', pd.DatetimeIndex(d[column]).year)
#     # d.insert(d.columns.get_loc(column), column+'_month', pd.DatetimeIndex(d[column]).month)
#     # d.insert(d.columns.get_loc(column), column+'_day', pd.DatetimeIndex(d[column]).day)
#     d.insert(d.columns.get_loc(column), column+'_hour', pd.DatetimeIndex(d[column]).hour)
#     d.insert(d.columns.get_loc(column), column+'_minute', pd.DatetimeIndex(d[column]).minute)
#     del d[column]

# date_time_extractor(df)
# df

# calc distance

In [464]:
from math import sin, cos, sqrt, atan2, radians

def distance_calculator(d):
  distances = []
  for index, record in d.iterrows():
    lat1 = radians(record['pick_lat'])
    lon1 = radians(record['pick_lon'])
    lat2 = radians(record['drop_lat'])
    lon2 = radians(record['drop_lon'])
    d_lat = lat2-lat1
    d_lon = lon2-lon1

    a = sin(d_lat/2)**2 +cos(lat1)*cos(lat2)*sin(d_lon/2)**2
    c = 2*atan2(sqrt(a), sqrt(1-a))

    distances.append(round(6373*c, 2))
  del d['pick_lat']
  del d['pick_lon']
  del d['drop_lat']
  del d['drop_lon']
  return distances

distances = distance_calculator(train_df)
train_df.insert(train_df.columns.get_loc('fare'), 'distance', distances)
train_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,distance,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,5.09,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,3.17,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.31,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,0.86,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,8.15,358.39,correct
...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,2.11,198.26,correct
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,10.87,581.23,correct
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,1.05,76.20,correct
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,2.88,133.31,correct


# fare per second and second per fare

In [465]:
# df.insert(df.columns.get_loc('meter_waiting_till_pickup'), 'meter_waiting_fare_rate', df['meter_waiting_fare']/df['meter_waiting'])
# df.insert(df.columns.get_loc('distance'), 'fare_per_km', df['fare']/df['distance'])
# df.insert(df.columns.get_loc('fare_per_km'), 'speed', df['distance']*1000/df['duration'])
train_df.insert(train_df.columns.get_loc('distance'), 'fare_per_second', train_df['fare']/train_df['duration'])
train_df.insert(train_df.columns.get_loc('distance'), 'seconds_per_fare', (train_df['duration']-train_df['meter_waiting'])/train_df['fare'])
train_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,fare_per_second,seconds_per_fare,distance,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,0.324125,2.878070,5.09,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,0.250126,3.760425,3.17,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,0.277498,3.338417,6.31,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,0.137625,3.973269,0.86,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,NaN,NaN,8.15,358.39,correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,0.236587,3.757692,2.11,198.26,correct
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,0.270214,2.964403,10.87,581.23,correct
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,0.289734,3.333333,1.05,76.20,correct
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,0.155373,5.573475,2.88,133.31,correct


# fare per dist and dist per fare

In [0]:
def get_fare_per_dist(df):
  df.insert(df.columns.get_loc('distance'), 'fare_per_dist', (df['fare']-(df['meter_waiting_fare']+df['additional_fare']))/df['distance'])
  return df
# df.insert(df.columns.get_loc('distance'), 'dist_per_fare', (df['distance'])/df['fare'])

In [467]:
train_df=get_fare_per_dist(train_df)
train_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,fare_per_second,seconds_per_fare,fare_per_dist,distance,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,0.324125,2.878070,51.045187,5.09,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,0.250126,3.760425,59.100946,3.17,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,0.277498,3.338417,46.139461,6.31,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,0.137625,3.973269,65.274651,0.86,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,NaN,NaN,NaN,8.15,358.39,correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,0.236587,3.757692,86.416161,2.11,198.26,correct
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,0.270214,2.964403,52.505060,10.87,581.23,correct
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,0.289734,3.333333,62.571429,1.05,76.20,correct
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,0.155373,5.573475,42.642361,2.88,133.31,correct


# correct replace 1

In [468]:
train_df.replace(to_replace=['correct', 'incorrect'], value=[1, 0], inplace=True)
train_df['label'].value_counts()

1    15495
0     1681
Name: label, dtype: int64

In [469]:
train_df = train_df.drop(['tripid'], axis=1)
# del df['pickup_time']
# del df['pick_lat']
# del df['pick_lon']
# del df['drop_lat']
# del df['drop_lon']
train_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,fare_per_second,seconds_per_fare,fare_per_dist,distance,fare,label
0,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,0.324125,2.878070,51.045187,5.09,270.32,1
1,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,0.250126,3.760425,59.100946,3.17,197.85,1
2,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,0.277498,3.338417,46.139461,6.31,301.64,1
3,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,0.137625,3.973269,65.274651,0.86,82.30,1
4,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,NaN,NaN,NaN,8.15,358.39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,0.236587,3.757692,86.416161,2.11,198.26,1
17172,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,0.270214,2.964403,52.505060,10.87,581.23,1
17173,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,0.289734,3.333333,62.571429,1.05,76.20,1
17174,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,0.155373,5.573475,42.642361,2.88,133.31,1


In [470]:
features_df=train_df.drop(columns=['pickup_time','drop_time','meter_waiting_till_pickup'])  
features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare_per_second,seconds_per_fare,fare_per_dist,distance,fare,label
0,10.5,834.0,56.0,0.0000,0.324125,2.878070,51.045187,5.09,270.32,1
1,10.5,791.0,47.0,0.0000,0.250126,3.760425,59.100946,3.17,197.85,1
2,10.5,1087.0,80.0,0.0000,0.277498,3.338417,46.139461,6.31,301.64,1
3,10.5,598.0,271.0,15.6638,0.137625,3.973269,65.274651,0.86,82.30,1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.15,358.39,1
...,...,...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,0.236587,3.757692,86.416161,2.11,198.26,1
17172,10.5,2151.0,428.0,0.0000,0.270214,2.964403,52.505060,10.87,581.23,1
17173,10.5,263.0,9.0,0.0000,0.289734,3.333333,62.571429,1.05,76.20,1
17174,10.5,858.0,115.0,0.0000,0.155373,5.573475,42.642361,2.88,133.31,1


In [0]:
# ## Correlation
# import seaborn as sns
# import matplotlib.pyplot as plt
# #get correlations of each features in dataset
# corrmat = df.corr()
# top_corr_features = corrmat.index
# plt.figure(figsize=(20,20))
# #plot heat map
# g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [472]:
cols = features_df.columns
features_df = features_df[cols[0:len(cols)-1]]
features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare_per_second,seconds_per_fare,fare_per_dist,distance,fare
0,10.5,834.0,56.0,0.0000,0.324125,2.878070,51.045187,5.09,270.32
1,10.5,791.0,47.0,0.0000,0.250126,3.760425,59.100946,3.17,197.85
2,10.5,1087.0,80.0,0.0000,0.277498,3.338417,46.139461,6.31,301.64
3,10.5,598.0,271.0,15.6638,0.137625,3.973269,65.274651,0.86,82.30
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.15,358.39
...,...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,0.236587,3.757692,86.416161,2.11,198.26
17172,10.5,2151.0,428.0,0.0000,0.270214,2.964403,52.505060,10.87,581.23
17173,10.5,263.0,9.0,0.0000,0.289734,3.333333,62.571429,1.05,76.20
17174,10.5,858.0,115.0,0.0000,0.155373,5.573475,42.642361,2.88,133.31


In [473]:
labels_df = train_df['label'].values
labels_df

array([1, 1, 1, ..., 1, 1, 1])

In [0]:
# from numpy import nan
# from numpy import isnan
# from sklearn.impute import SimpleImputer

# imp_features = [['additional_fare', 'duration', 'meter_waiting', 'fare']]
# imp_features2 = [['meter_waiting_fare', 'meter_waiting_till_pickup']]

# for feature in imp_features2:
#   imputer = SimpleImputer(missing_values=nan, strategy='most_frequent')
#   X[feature] = imputer.fit_transform(X[feature].values)

# for feature in imp_features:
#   imputer = SimpleImputer(missing_values=nan, strategy='mean')
#   X[feature] = imputer.fit_transform(X[feature].values)
# X

In [475]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_df, labels_df, test_size=0.2, random_state=6)
X_train

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare_per_second,seconds_per_fare,fare_per_dist,distance,fare
16460,10.5,1348.0,117.0,0.0000,0.264874,3.447696,45.598684,7.60,357.05
9339,10.5,228.0,18.0,0.0000,3.777544,0.243823,680.624000,1.25,861.28
12446,10.5,306.0,82.0,0.0000,0.205719,3.558380,49.481132,1.06,62.95
1883,10.5,820.0,80.0,0.0000,0.283610,3.181975,57.231959,3.88,232.56
6100,10.5,138.0,8.0,0.4664,0.441739,2.132546,142.838857,0.35,60.96
...,...,...,...,...,...,...,...,...,...
7243,40.5,1519.0,259.0,15.0997,0.190829,4.346776,96.012418,2.44,289.87
8527,10.5,767.0,10.0,0.0000,0.284146,3.473433,52.516456,3.95,217.94
4714,10.5,483.0,2.0,0.0000,0.329524,3.022116,45.461774,3.27,159.16
10196,10.5,413.0,53.0,0.0000,0.226077,3.855628,48.180233,1.72,93.37


In [0]:
# params={
#  "learning_rate"    : [ 0.10, 0.20, 0.30, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9 ] ,
#  "max_depth"        : [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
#  "min_child_weight" : [ 1, 2, 3, 4, 5, 6, 7, 8, 9 ],
#  "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9 ],
#  "colsample_bytree" : [ 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9 ]   
# }

# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# import xgboost

# classifier=xgboost.XGBClassifier()
# random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)
# random_search.fit(X,y)

In [0]:
# random_search.best_estimator_

In [0]:
# random_search.best_params_

In [0]:
# !pip install catboost

In [480]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

for i in range(1,20):
  l_rate= i*0.05
  model = XGBClassifier(learning_rate=l_rate)
  # model.fit(X, y)
  model.fit(X_train, y_train)
  pred=model.predict(X_test)
  print(roc_auc_score(y_test, pred))
  # 0.8367421259440911---9

0.8047440192266104
0.819235383165291
0.8235319919861921
0.829100649999796
0.8348294617609976
0.843903140645593
0.833557412568295
0.8311642871424083
0.8314845946376036
0.8356210497109073
0.8196974828319263
0.8333880780326182
0.831324440890006
0.8231933229148389
0.8252661408455302
0.8246347066432183
0.8235136304100343
0.8333880780326182
0.8265381900382328


In [481]:
model = XGBClassifier()
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
cv_results = cross_val_score(model, features_df, labels_df, cv=kfold, scoring='f1')
cv_results

array([0.97268107, 0.97862839, 0.97228417, 0.9714467 , 0.9789675 ,
       0.97331639, 0.98090388, 0.97765006, 0.97895408, 0.97550111])

In [482]:
cv_results.mean()
# X.columns

0.9760333352470797

In [483]:
model = XGBClassifier(learning_rate=0.2)
model.fit(X_train, y_train)
imp=pd.DataFrame()
imp['feature']=X.columns
imp['importance']=model.feature_importances_
imp
  

,feature,importance
0,additional_fare,0.089605
1,duration,0.054432
2,meter_waiting,0.028554
3,meter_waiting_fare,0.048840
4,fare_per_second,0.051530
5,seconds_per_fare,0.396094
6,fare_per_dist,0.087240
7,distance,0.037109
8,fare,0.206596


In [0]:
# from sklearn.metrics import jaccard_similarity_score
# from sklearn.metrics import f1_score
# from sklearn.metrics import log_loss
# from sklearn.model_selection import cross_val_score
# from sklearn.neighbors import KNeighborsClassifier

# X_train, y_train = X,y

# bestScore=0.0
# accList=[]
# for k in range(1,15):
#     clf_knn = KNeighborsClassifier(n_neighbors=k,algorithm='auto')
    
#     scores = cross_val_score(clf_knn, X_train, y_train, cv=10)
#     score=scores.mean()
#     accList.append(score)
    
#     if score > bestScore:
#         bestScore=score
#         best_clf=clf_knn
#         bestK=k
# print("Best K is :",bestK,"| Cross validation Accuracy :",bestScore)
# clf_knn=best_clf

In [0]:
# clf_knn.fit(X_train,y_train)
# y_pred=best_clf.predict(X_train)
# trainScores={}

In [0]:
# trainScores['KNN-jaccard']=jaccard_similarity_score(y_train, y_pred)
# trainScores['KNN-f1-score']=f1_score(y_train, y_pred, average='weighted')
# trainScores

In [0]:
# plt.plot(range(1,15),accList)
# plt.xlabel('K')
# plt.ylabel('CV Accuracy')
# plt.show()

In [0]:
# from sklearn.metrics import roc_auc_score

# pred=model.predict(X_test)
# roc_auc_score(y_test, pred)

In [489]:
test_df = pd.read_csv('/content/drive/My Drive/ML Project/test.csv')
test_X = test_df[test_df.columns]
test_X
# test_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
0,213284604,10.5,924,42,2.44860,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
1,213286352,10.5,4249,20,0.00000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
2,213293973,10.5,1552,255,2.65880,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
3,213294622,10.5,462,16,0.00000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
4,213298687,10.5,814,392,12.36920,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8571,222856243,10.5,1723,429,24.83332,3,3/16/2020 21:28,3/16/2020 21:56,6.85103,79.9567,6.85588,79.9214,388.48
8572,222857785,10.5,1378,80,0.00000,125,3/16/2020 21:59,3/16/2020 22:22,6.91293,79.9656,6.92112,79.8980,379.85
8573,222858416,10.5,418,56,3.28440,93,3/16/2020 22:02,3/16/2020 22:09,6.85718,79.9081,6.83868,79.9083,112.79
8574,222858691,10.5,1604,548,31.67440,17,3/16/2020 22:07,3/16/2020 22:34,6.91289,79.8846,6.93159,79.9145,248.46


In [490]:
# date_time_extractor(test_X)
dist = distance_calculator(test_X)
test_X.insert(test_X.columns.get_loc('fare'), 'distance', dist)
del test_X['tripid']
# test_X.insert(test_X.columns.get_loc('meter_waiting_till_pickup'), 'meter_waiting_fare_rate', test_X['meter_waiting_fare']/test_X['meter_waiting'])

# test_X.insert(test_X.columns.get_loc('distance'), 'fare_per_km', test_X['fare']/test_X['distance'])
# test_X.insert(test_X.columns.get_loc('fare_per_km'), 'speed', test_X['distance']*1000/test_X['duration'])
test_X.insert(test_X.columns.get_loc('distance'), 'fare_per_second', test_X['fare']/test_X['duration'])
test_X.insert(test_X.columns.get_loc('distance'), 'seconds_per_fare', (test_X['duration']-test_X['meter_waiting'])/test_X['fare'])
get_fare_per_dist(test_X)
test_X

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,fare_per_second,seconds_per_fare,fare_per_dist,distance,fare
0,10.5,924,42,2.44860,148,2/1/2020 0:38,2/1/2020 0:53,0.313063,3.049055,41.180537,6.71,289.27
1,10.5,4249,20,0.00000,91,2/1/2020 1:02,2/1/2020 2:13,0.450153,2.211011,45.758961,41.57,1912.70
2,10.5,1552,255,2.65880,23,2/1/2020 5:02,2/1/2020 5:28,0.253866,3.291878,64.331284,5.92,394.00
3,10.5,462,16,0.00000,198,2/1/2020 5:30,2/1/2020 5:38,0.334026,2.890098,43.581818,3.30,154.32
4,10.5,814,392,12.36920,69,2/1/2020 7:00,2/1/2020 7:14,0.181167,2.861599,48.108417,2.59,147.47
...,...,...,...,...,...,...,...,...,...,...,...,...
8571,10.5,1723,429,24.83332,3,3/16/2020 21:28,3/16/2020 21:56,0.225467,3.330931,89.631137,3.94,388.48
8572,10.5,1378,80,0.00000,125,3/16/2020 21:59,3/16/2020 22:22,0.275653,3.417138,49.115691,7.52,379.85
8573,10.5,418,56,3.28440,93,3/16/2020 22:02,3/16/2020 22:09,0.269833,3.209504,48.060971,2.06,112.79
8574,10.5,1604,548,31.67440,17,3/16/2020 22:07,3/16/2020 22:34,0.154900,4.250181,52.893744,3.90,248.46


In [491]:
test_X=test_X.drop(columns=['pickup_time','drop_time','meter_waiting_till_pickup'])  
test_X.columns

Index(['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'fare_per_second', 'seconds_per_fare', 'fare_per_dist', 'distance',
       'fare'],
      dtype='object')

In [492]:
test_X


,additional_fare,duration,meter_waiting,meter_waiting_fare,fare_per_second,seconds_per_fare,fare_per_dist,distance,fare
0,10.5,924,42,2.44860,0.313063,3.049055,41.180537,6.71,289.27
1,10.5,4249,20,0.00000,0.450153,2.211011,45.758961,41.57,1912.70
2,10.5,1552,255,2.65880,0.253866,3.291878,64.331284,5.92,394.00
3,10.5,462,16,0.00000,0.334026,2.890098,43.581818,3.30,154.32
4,10.5,814,392,12.36920,0.181167,2.861599,48.108417,2.59,147.47
...,...,...,...,...,...,...,...,...,...
8571,10.5,1723,429,24.83332,0.225467,3.330931,89.631137,3.94,388.48
8572,10.5,1378,80,0.00000,0.275653,3.417138,49.115691,7.52,379.85
8573,10.5,418,56,3.28440,0.269833,3.209504,48.060971,2.06,112.79
8574,10.5,1604,548,31.67440,0.154900,4.250181,52.893744,3.90,248.46


In [0]:
# test_X['drop_time'] = pd.to_datetime(test_X['drop_time'])
# test_X['pickup_time'] = pd.to_datetime(test_X['pickup_time'])

In [0]:
# imp_features = [['additional_fare','duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','pick_lat','pick_lon','drop_lat','drop_lon','fare']]

# for feature in imp_features:
#   imputer = SimpleImputer(missing_values=nan, strategy='mean')
#   transformed_values = imputer.fit_transform(test_df[feature].values)
# test_df

In [0]:
# test_df['duration'] = test_df['duration'].astype(np.float64)
# test_df['meter_waiting'] = test_df['meter_waiting'].astype(np.float64)
# test_df['meter_waiting_till_pickup'] = test_df['meter_waiting_till_pickup'].astype(np.float64)
# test_df

In [0]:
# test_df['label'] = pd.to_numeric(test_df['label'])
# test_df

In [0]:
# xgb_pred = model.predict(test_X)
# test_df['prediction'] = xgb_pred
# test_df

In [498]:
for i in range(1,20):
  l_rate= i*0.05
  model = XGBClassifier(learning_rate=l_rate)
  model.fit(features_df, labels_df)
  preds = model.predict(test_X)
  test_df['prediction'] = preds
  print(test_df['prediction'].value_counts())

1    8205
0     371
Name: prediction, dtype: int64
1    8159
0     417
Name: prediction, dtype: int64
1    8140
0     436
Name: prediction, dtype: int64
1    8127
0     449
Name: prediction, dtype: int64
1    8128
0     448
Name: prediction, dtype: int64
1    8132
0     444
Name: prediction, dtype: int64
1    8126
0     450
Name: prediction, dtype: int64
1    8137
0     439
Name: prediction, dtype: int64
1    8113
0     463
Name: prediction, dtype: int64
1    8112
0     464
Name: prediction, dtype: int64
1    8116
0     460
Name: prediction, dtype: int64
1    8112
0     464
Name: prediction, dtype: int64
1    8115
0     461
Name: prediction, dtype: int64
1    8103
0     473
Name: prediction, dtype: int64
1    8101
0     475
Name: prediction, dtype: int64
1    8102
0     474
Name: prediction, dtype: int64
1    8092
0     484
Name: prediction, dtype: int64
1    8118
0     458
Name: prediction, dtype: int64
1    8095
0     481
Name: prediction, dtype: int64


In [505]:
model = XGBClassifier(learning_rate=0.2)
model.fit(features_df, labels_df)
preds = model.predict(test_X)
test_df['prediction'] = preds
test_df['prediction'].value_counts()

1    8127
0     449
Name: prediction, dtype: int64

In [506]:
features_df
test_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,prediction
0,213284604,10.5,924,42,2.44860,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27,1
1,213286352,10.5,4249,20,0.00000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70,1
2,213293973,10.5,1552,255,2.65880,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00,1
3,213294622,10.5,462,16,0.00000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32,1
4,213298687,10.5,814,392,12.36920,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8571,222856243,10.5,1723,429,24.83332,3,3/16/2020 21:28,3/16/2020 21:56,6.85103,79.9567,6.85588,79.9214,388.48,1
8572,222857785,10.5,1378,80,0.00000,125,3/16/2020 21:59,3/16/2020 22:22,6.91293,79.9656,6.92112,79.8980,379.85,1
8573,222858416,10.5,418,56,3.28440,93,3/16/2020 22:02,3/16/2020 22:09,6.85718,79.9081,6.83868,79.9083,112.79,1
8574,222858691,10.5,1604,548,31.67440,17,3/16/2020 22:07,3/16/2020 22:34,6.91289,79.8846,6.93159,79.9145,248.46,1


In [507]:
submission_df = test_df[['tripid','prediction']]
submission_df

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1
...,...,...
8571,222856243,1
8572,222857785,1
8573,222858416,1
8574,222858691,1


In [0]:
submission_df.to_csv('submission_34.csv', index=True)